# Set Up

In [1]:
from great_expectations.data_context import FileDataContext

context = FileDataContext.create(project_root_dir='./')

In [2]:
datasource_name = 'csv-cust-analyst'
datasource = context.sources.add_pandas(datasource_name)

asset_name = 'cust-analyst'
path_to_data = '/BootcampHacktiv8/Milestone/Milestone_Phase_2/p2-ftds008-hck-m3-Yobi-Aditias/P2M3_yobi_ad_data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()  

In [3]:
# Creat an expectation suite
expectation_suite_name = 'expectation-cust-analyst'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,id,year_birth,education,marital_status,income,kidhome,teenhome,dt_customer,recency,mntwines,...,numwebvisitsmonth,acceptedcmp3,acceptedcmp4,acceptedcmp5,acceptedcmp1,acceptedcmp2,complain,z_costcontact,z_revenue,response_promo
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,7,menolak,menolak,menolak,menolak,menolak,tidak komplain,3,11,menerima
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,5,menolak,menolak,menolak,menolak,menolak,tidak komplain,3,11,menolak
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,menolak,menolak,menolak,menolak,menolak,tidak komplain,3,11,menolak
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,6,menolak,menolak,menolak,menolak,menolak,tidak komplain,3,11,menolak
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,5,menolak,menolak,menolak,menolak,menolak,tidak komplain,3,11,menolak


# TES 1. Chek tipe kolom

kolom jumlah pembelian mestinya bilangann bulan, maka di chek tipe kolom

In [4]:
validator.expect_column_values_to_be_in_type_list('numdealspurchases', ['integer'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": false,
  "result": {
    "observed_value": "int64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# TES 2. chek unik value

kolom id merupakan kolom yang seharusnya unique, maka dari itu akan dilakukan chek

In [5]:
validator.expect_column_values_to_be_unique('id')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 2240,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# TES 3. Chek apakah nilai dari kolom melebihi ambang batas atau tidak

tahun lahir customer mestinya tidak memiliki nilai angka yang aneh, maka akan di chek

In [6]:
validator.expect_column_values_to_be_between(
    column='year_birth', min_value=1900, max_value=2023
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": false,
  "result": {
    "element_count": 2240,
    "unexpected_count": 2,
    "unexpected_percent": 0.08928571428571429,
    "partial_unexpected_list": [
      1893,
      1899
    ],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.08928571428571429,
    "unexpected_percent_nonmissing": 0.08928571428571429
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# TES 4. chek apakah kolom hanya mengandung nilai tertentu

kolom komplain mestinya berisi value berupa komplain dan tidak komplain saja, tidak lebih dan tidak kurang, maka akan di chek

In [23]:
validator.expect_column_values_to_be_in_set('complain', ['komplain', 'tidak komplain'])

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 2240,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# TES 5. (tes yang tidak dicontoh di kelas no. 1) chek nilai minimum yang dimiliki kolom

chek nilai minimum pada kolom jumlah anak di rumah, untuk data yang banyak nilai minimum setidaknya ialah 0, sekaligus untuk memastikan bahwa ada pembeli yang belum memiliki anak baik itu lajang ataupun sudah menikah

In [34]:
validator.expect_column_min_to_be_between('kidhome',max_value=0)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# TES 6. (tes yang tidak dicontoh di kelas no. 2) chek nilai maksimum yang dimiliki kolom

setidaknya akan ada beberapa pembeli yang nilai pembelian katalognya lebih dari 20, maka dari itu akan di chek apakah ada atau tidak pembeli yang membeli lebih dari 20

In [40]:

validator.expect_column_max_to_be_between('numcatalogpurchases', min_value =20, max_value=50)


Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 28
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# TES 7. (tes yang tidak dicontoh di kelas no. 3) chek nilai mean yang dimiliki kolom

rata-rata pembeli mengunjungi website setidaknya sekitar 4-6 kali sebulan, untuk memastikan hal tersebut maka dilaksanakan tes ini

In [39]:
validator.expect_column_mean_to_be_between('numwebvisitsmonth', min_value=4, max_value=6)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 5.316517857142857
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}